In [1]:
import sys
import rp
import torch
import numpy as np
from einops import rearrange
import shlex

# Global config
IN_NOTEBOOK = rp.running_in_jupyter_notebook()
top_dir = rp.get_git_working_dir('.')
ltx_src = rp.path_join(top_dir, 'LTX2', 'src')

# Add LTX packages to path
for pkg in ['ltx-core', 'ltx-trainer', 'ltx-pipelines']:
    sys.path.insert(0, rp.path_join(ltx_src, 'packages', pkg, 'src'))

from ltx_trainer.model_loader import load_video_vae_encoder, load_video_vae_decoder

print(f"Running in notebook: {IN_NOTEBOOK}")
print(f"Top directory: {top_dir}")

Running in notebook: True
Top directory: /root/CleanCode/Experiments/VideoVaeTests


In [ ]:
models_dir = rp.path_join(top_dir, 'LTX2', 'models')
vae_path = rp.path_join(models_dir, 'ltx-2-19b-distilled.safetensors')
device = rp.select_torch_device(prefer_used=True, reserve=True)
dtype = torch.bfloat16

print(f"Device: {device}, dtype: {dtype}")
print("Loading VAE...")
vae_encoder = load_video_vae_encoder(vae_path, device=device, dtype=dtype)
vae_decoder = load_video_vae_decoder(vae_path, device=device, dtype=dtype)
print("VAE loaded!")

rp._run_sys_command(f"rclone {shlex.quote()}")